In [2]:
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib

# S-Bahn Stations

In [3]:
# get the response in the form of html
wikiurl_sbahns="https://de.wikipedia.org/wiki/Liste_der_Stationen_der_S-Bahn_Berlin"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl_sbahns)
print(response.status_code)

200


In [4]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.content, 'html.parser')

indiatable=soup.find('table', {'class':"wikitable"})

In [5]:
df_sbahns=pd.read_html(str(indiatable))
# convert list to dataframe
df_sbahns=pd.DataFrame(df_sbahns[0])
print(df_sbahns.head())

                         S-Bahnhof(ehem. Name)         Strecke  Linie(n)  \
0  Adlershof(bis 1935 Adlershof=Alt-Glienicke)  Görlitzer Bahn       NaN   
1                                  Ahrensfelde  Wriezener Bahn       NaN   
2                                 Albrechtshof  Hamburger Bahn       NaN   
3                               Alexanderplatz       Stadtbahn       NaN   
4    Alt-Reinickendorf(bis 1994 Reinickendorf)  Kremmener Bahn       NaN   

                                                Lage  \
0  Adlershof(Treptow-Köpenick)52° 26′ 5″ N, 13° 3...   
1  Marzahn(Marzahn-Hellersdorf)52° 34′ 18″ N, 13°...   
2        Staaken(Spandau)52° 32′ 58″ N, 13° 7′ 42″ O   
3           Mitte(Mitte)52° 31′ 17″ N, 13° 24′ 43″ O   
4  Reinickendorf(Reinickendorf)52° 34′ 40″ N, 13°...   

                    Aufnahme   Einstellung Unnamed: 6 Unnamed: 7 Bemerkungen  \
0               6. Nov. 1928           NaN        NaN        NaN         NaN   
1              30. Dez. 1982           NaN    

In [16]:
sstat = []
for station in indiatable.find_all('tr')[1:]:
    stat= station.select("td")[2]
    lines =[]
    for img in stat.select('img'):
        lin = img['alt']
        lines.append(lin)
    sstat.append(lines)
len(sstat)

193

In [23]:
sstat_new = []
for sublist in sstat:
    if len(sublist) == 0:
        sstat_new.append(np.nan)
    else:
        sstat_new.append(sublist)
        
sstat_new

[['S45', 'S46', 'S8', 'S85', 'S9'],
 ['S7'],
 nan,
 ['S3', 'S5', 'S7', 'S9'],
 ['S25'],
 ['S45', 'S9'],
 ['S1', 'S2', 'S25', 'S26'],
 ['S2'],
 ['S7'],
 ['S45', 'S46', 'S47', 'S8', 'S85', 'S9'],
 ['S3', 'S5', 'S7', 'S9'],
 ['S8'],
 ['S2'],
 ['S2'],
 ['S3'],
 ['S41', 'S42'],
 ['S5'],
 ['S5'],
 ['S1', 'S8'],
 ['S2', 'S8'],
 ['S2'],
 ['S1'],
 ['S1', 'S2', 'S25', 'S26', 'S8', 'S85'],
 ['S1'],
 ['S1', 'S2', 'S25', 'S26'],
 ['S2'],
 ['S2'],
 ['S41', 'S42', 'S46'],
 ['S3', 'S5', 'S7', 'S9'],
 nan,
 nan,
 nan,
 nan,
 ['S25'],
 nan,
 ['S46', 'S8'],
 ['S3'],
 nan,
 ['S1'],
 ['S45', 'S9'],
 ['S45', 'S9'],
 ['S41', 'S42', 'S8', 'S85'],
 ['S5'],
 ['S1'],
 ['S5', 'S7', 'S75'],
 ['S3'],
 ['S1', 'S2', 'S25', 'S26', 'S3', 'S5', 'S7', 'S9'],
 ['S1'],
 nan,
 ['S75'],
 ['S1', 'S2', 'S25', 'S26', 'S41', 'S42'],
 ['S41', 'S42', 'S8', 'S85'],
 ['S7'],
 ['S46', 'S8', 'S85'],
 ['S45', 'S9'],
 ['S7'],
 ['S3', 'S5', 'S7', 'S9'],
 ['S41', 'S42', 'S46'],
 ['S3', 'S5', 'S7', 'S9'],
 ['S3', 'S9'],
 ['S5'],
 ['S41', '

In [24]:
df_sbahns["Linie(n)"] = sstat_new
df_sbahns

,S-Bahnhof(ehem. Name),Strecke,Linie(n),Lage,Aufnahme,Einstellung,Unnamed: 6,Unnamed: 7,Bemerkungen,Bild
0,Adlershof(bis 1935 Adlershof=Alt-Glienicke),Görlitzer Bahn,"[S45, S46, S8, S85, S9]","Adlershof(Treptow-Köpenick)52° 26′ 5″ N, 13° 3...",6. Nov. 1928,NaN,NaN,NaN,NaN,NaN
1,Ahrensfelde,Wriezener Bahn,[S7],"Marzahn(Marzahn-Hellersdorf)52° 34′ 18″ N, 13°...",30. Dez. 1982,NaN,×,NaN,NaN,NaN
2,Albrechtshof,Hamburger Bahn,NaN,"Staaken(Spandau)52° 32′ 58″ N, 13° 7′ 42″ O",14. Aug. 1951,9. Okt. 1961,NaN,NaN,NaN,NaN
3,Alexanderplatz,Stadtbahn,"[S3, S5, S7, S9]","Mitte(Mitte)52° 31′ 17″ N, 13° 24′ 43″ O",11. Juni 1928,NaN,×,×,NaN,NaN
4,Alt-Reinickendorf(bis 1994 Reinickendorf),Kremmener Bahn,[S25],"Reinickendorf(Reinickendorf)52° 34′ 40″ N, 13°...",16. März 192728. Mai 1995,9. Jan. 1984,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
188,Zehlendorf(bis 1938 Zehlendorf Mitte),StammbahnWannseebahn,[S1],"Zehlendorf(Steglitz-Zehlendorf)52° 25′ 51″ N, ...",15. Mai 19331. Feb. 1985,18. Sep. 1980,NaN,NaN,1980 infolge des Reichsbahnerstreiks geschlossen,NaN
189,Zehlendorf Süd,Stammbahn,NaN,"Zehlendorf(Steglitz-Zehlendorf)52° 25′ 32″ N, ...",20. Dez. 1972,18. Sep. 1980,NaN,NaN,NaN,NaN
190,Zepernick (b Bernau)(bis 1960 Zepernick [Kr Ni...,Stettiner Bahn,[S2],"Panketal52° 39′ 35″ N, 13° 32′ 2″ O",8. Aug. 1924,NaN,NaN,NaN,NaN,NaN
191,Zeuthen,Görlitzer Bahn,[S46],"Zeuthen52° 20′ 55″ N, 13° 37′ 39″ O",30. Apr. 1951,NaN,NaN,NaN,NaN,NaN


In [66]:
# Create a new df
data_sbahns = df_sbahns[["S-Bahnhof(ehem. Name)", "Linie(n)", "Lage"]]

# Drop stations that were closed (marked with red & have 0 lines)
data_sbahns = data_sbahns[data_sbahns['Linie(n)'].notna()]

# Drop stations that were not yet opened (marked with green (Perleberger Brücke))
data_sbahns = data_sbahns[data_sbahns["S-Bahnhof(ehem. Name)"]!= "Perleberger Brücke"]

# Reset index
data_sbahns.reset_index(inplace = True)

# Rename columns
# Shorten station names
# Split Area - Longitude - Latitude

In [67]:
data_sbahns

,index,S-Bahnhof(ehem. Name),Linie(n),Lage
0,0,Adlershof(bis 1935 Adlershof=Alt-Glienicke),"[S45, S46, S8, S85, S9]","Adlershof(Treptow-Köpenick)52° 26′ 5″ N, 13° 3..."
1,1,Ahrensfelde,[S7],"Marzahn(Marzahn-Hellersdorf)52° 34′ 18″ N, 13°..."
2,3,Alexanderplatz,"[S3, S5, S7, S9]","Mitte(Mitte)52° 31′ 17″ N, 13° 24′ 43″ O"
3,4,Alt-Reinickendorf(bis 1994 Reinickendorf),[S25],"Reinickendorf(Reinickendorf)52° 34′ 40″ N, 13°..."
4,5,Altglienicke,"[S45, S9]","Altglienicke(Treptow-Köpenick)52° 24′ 26″ N, 1..."
...,...,...,...,...
163,187,Yorckstraße (Großgörschenstraße),[S1],"Schöneberg(Tempelhof-Schöneberg)52° 29′ 32″ N,..."
164,188,Zehlendorf(bis 1938 Zehlendorf Mitte),[S1],"Zehlendorf(Steglitz-Zehlendorf)52° 25′ 51″ N, ..."
165,190,Zepernick (b Bernau)(bis 1960 Zepernick [Kr Ni...,[S2],"Panketal52° 39′ 35″ N, 13° 32′ 2″ O"
166,191,Zeuthen,[S46],"Zeuthen52° 20′ 55″ N, 13° 37′ 39″ O"


# U-Bahn Stations

In [2]:
# get the response in the form of html
wikiurl_ubahns="https://de.wikipedia.org/w/index.php?title=Liste_der_Berliner_U-Bahnh%C3%B6fe&veaction=edit&section=1"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl_ubahns)
print(response.status_code)

200


In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
indiatable=soup.find_all('table',{'class':"wikitable"})


In [5]:
indiatable[1]

<table class="wikitable sortable" style="width: 100%; font-size: 90%;">
<tbody><tr>
<th style="width:15%">Bahnhof (Kürzel)<br/> Karte
</th>
<th style="width:3%">Linie
</th>
<th style="width:8%">Eröffnung
</th>
<th style="width:5%">Lage
</th>
<th style="width:9%">Ortsteil
</th>
<th style="width:2%">Umstieg
</th>
<th style="width:2%">Denkmal
</th>
<th class="unsortable" style="width:20%">Anmerkungen
</th>
<th class="unsortable" style="width:22%">Sehenswürdigkeiten
</th>
<th class="unsortable" style="width:10%">Bild
</th></tr>
<tr id="A">
<td data-sort-value="Adenauerplatz"><a href="/wiki/U-Bahnhof_Adenauerplatz" title="U-Bahnhof Adenauerplatz">Adenauerplatz</a> (Ado)<br/> <small><span class="plainlinks-print" id="Adenauerplatz" title="Koordinate"><a class="external text" href="https://geohack.toolforge.org/geohack.php?pagename=Liste_der_Berliner_U-Bahnh%C3%B6fe&amp;language=de&amp;params=52.499722_N_13.307222_E_region:DE-BE_type:landmark&amp;title=Adenauerplatz">52° 29′ 59″ N, 13° 18′ 26

In [23]:
stations = indiatable[1].find_all('tr')[1:]

In [28]:
ustat = []
for station in indiatable[1].find_all('tr')[1:]:
    stat= station.select("td")[1]["data-sort-value"]
    ustat.append(stat)

len(ustat)

200

In [6]:
df_ubahns=pd.read_html(str(indiatable[1]))
# convert list to dataframe
df_ubahns=pd.DataFrame(df_ubahns[0])
print(df_ubahns.head())

                              Bahnhof (Kürzel) Karte  Linie      Eröffnung  \
0   Adenauerplatz (Ado) 52° 29′ 59″ N, 13° 18′ 26″ O    NaN  28. Apr. 1978   
1  Afrikanische Straße (Afr) 52° 33′ 38″ N, 13° 2...    NaN    3. Mai 1956   
2    Alexanderplatz (A) 52° 31′ 17″ N, 13° 24′ 48″ O    NaN   1. Juli 1913   
3   Alexanderplatz (Al) 52° 31′ 17″ N, 13° 24′ 48″ O    NaN  21. Dez. 1930   
4   Alexanderplatz (Ap) 52° 31′ 17″ N, 13° 24′ 48″ O    NaN  18. Apr. 1930   

     Lage        Ortsteil  Umstieg  \
0  Tunnel  Charlottenburg      NaN   
1  Tunnel         Wedding      NaN   
2  Tunnel           Mitte      NaN   
3  Tunnel           Mitte      NaN   
4  Tunnel           Mitte      NaN   

                                      Denkmal                 Anmerkungen  \
0                                           –                         NaN   
1                                           –                         NaN   
2  Eintrag in der Berliner Landesdenkmalliste                         N

In [32]:
df_ubahns["Linie"] = ustat

In [36]:
data_ubahns = df_ubahns[["Bahnhof (Kürzel) Karte", "Linie", "Ortsteil"]]
data_ubahns

,Bahnhof (Kürzel) Karte,Linie,Ortsteil
0,"Adenauerplatz (Ado) 52° 29′ 59″ N, 13° 18′ 26″ O",U7,Charlottenburg
1,"Afrikanische Straße (Afr) 52° 33′ 38″ N, 13° 2...",U6,Wedding
2,"Alexanderplatz (A) 52° 31′ 17″ N, 13° 24′ 48″ O",U2,Mitte
3,"Alexanderplatz (Al) 52° 31′ 17″ N, 13° 24′ 48″ O",U5,Mitte
4,"Alexanderplatz (Ap) 52° 31′ 17″ N, 13° 24′ 48″ O",U8,Mitte
...,...,...,...
195,"Yorckstraße (Y) 52° 29′ 35″ N, 13° 22′ 15″ O",U7,Schöneberg
196,"Zitadelle (Zi) 52° 32′ 16″ N, 13° 13′ 4″ O",U7,Haselhorst
197,"Zoologischer Garten (Zo) 52° 30′ 26″ N, 13° 19...",U2,Charlottenburg
198,"Zoologischer Garten (Zu) 52° 30′ 26″ N, 13° 19...",U9,Charlottenburg


In [ ]:
print('hello world')